In [79]:
#API key - eq63r7svaqsta7wm_6bzrykdx2zj4jfnh
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime
import pickle # pip install pickle
import re

In [30]:
#Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [39]:
#limiting only to cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

,marketSegment,name,slug,ticker,totalSupply
1748,Cryptocurrency,Dimecoin,dimecoin,DIME,5.559981e+11
1181,Cryptocurrency,Mooncoin,mooncoin,MOON,2.274909e+11
1854,Cryptocurrency,NewYorkCoin,newyorkcoin,NYC,1.437611e+11
1707,Cryptocurrency,Dogecoin,dogecoin,DOGE,1.291413e+11
1769,Cryptocurrency,eXPerience Chain,experience-chain,XPC,1.132155e+11
...,...,...,...,...,...
1105,Cryptocurrency,White Standard,white-standard,WSD,4.394020e+05
1442,Cryptocurrency,ParallelCoin,parallelcoin,DUO,3.147186e+05
347,Cryptocurrency,Unobtanium,unobtanium,UNO,2.033284e+05
1761,Cryptocurrency,Amoveo,amoveo,VEO,6.983818e+04


In [112]:
def metriclist(metric, slug):
     #change metric here. Use san.available_metrics() or san.available_metrics_for_slug(slug)
    fullname = metric + "/" + slug 
    metrics = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1d" #Manipulate this
    )
    newname = slug+"_"+metric
    finaldf = metrics.rename(columns={"value": newname})
    finaldf.index.name = 'datetime'
    finaldf.reset_index(inplace=True)
    finaldf['strdatetime'] = finaldf['datetime'].astype(str)
    return(finaldf)

In [113]:
#Change metric and slug fullname = metric + slug
#Running through all cryptocurrencies and adding price and daily returns
count = 0
metric_df = pd.DataFrame()
metric = "dev_activity"
for i in crypto['slug'].tolist():
    holddf = metriclist(metric, i)
    if count == 0:
        metric_df = holddf
    else:
        metric_df = pd.merge(metric_df, holddf, how = "left", on = "strdatetime")
    count = count+1

In [114]:
metric_df = metric_df.rename(columns = {"datetime_x": "datetime"})
metric_df = metric_df.loc[:,~metric_df.columns.duplicated()]

In [115]:
badnames = []
for i in metric_df.columns.tolist():
    if re.findall(metric, i):
        pass
    elif i == 'datetime':
        pass
    else:
        badnames.append(i)


In [116]:
metric_df = metric_df.drop(badnames, axis = 1)

In [117]:
metric_df.set_index('datetime', inplace=True)

In [123]:
a = metric_df.unstack().reset_index()
a = a.rename(columns = {"level_0":"Coin", 0:metric})
a

,Coin,datetime,dev_activity
0,dimecoin_dev_activity,2018-04-01 00:00:00+00:00,0.0
1,dimecoin_dev_activity,2018-04-02 00:00:00+00:00,13.0
2,dimecoin_dev_activity,2018-04-03 00:00:00+00:00,15.0
3,dimecoin_dev_activity,2018-04-04 00:00:00+00:00,0.0
4,dimecoin_dev_activity,2018-04-05 00:00:00+00:00,6.0
...,...,...,...
5480,experience-chain_dev_activity,2021-03-28 00:00:00+00:00,0.0
5481,experience-chain_dev_activity,2021-03-29 00:00:00+00:00,0.0
5482,experience-chain_dev_activity,2021-03-30 00:00:00+00:00,0.0
5483,experience-chain_dev_activity,2021-03-31 00:00:00+00:00,0.0


In [125]:
coinlist = []
for i in a['Coin'].tolist():
    coinlist.append(i.replace("_"+metric, ''))
a["Coin"] = coinlist

In [126]:
a

,Coin,datetime,dev_activity
0,dimecoin,2018-04-01 00:00:00+00:00,0.0
1,dimecoin,2018-04-02 00:00:00+00:00,13.0
2,dimecoin,2018-04-03 00:00:00+00:00,15.0
3,dimecoin,2018-04-04 00:00:00+00:00,0.0
4,dimecoin,2018-04-05 00:00:00+00:00,6.0
...,...,...,...
5480,experience-chain,2021-03-28 00:00:00+00:00,0.0
5481,experience-chain,2021-03-29 00:00:00+00:00,0.0
5482,experience-chain,2021-03-30 00:00:00+00:00,0.0
5483,experience-chain,2021-03-31 00:00:00+00:00,0.0


In [127]:
a.to_pickle(metric + "_daily.pkl")